# Fitting Powerlaws

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from lsoc.powerlaw import fit, data, vis


# Load Data



In [ ]:
model = "160m"
df_llc_nb10, df_loss = data.load_dfs(f"{model}-nb")
df_llc_orig, _ = data.load_dfs(model)


In [ ]:
# Just plot the differences?
analysis_interval = 2000, 70000

fig = go.Figure()
tasks = df_llc_nb10.columns
colors = vis.assign_cols(tasks)
for task in tasks:
    col = colors[task]
    faded = vis.fade(col)
    full_curve = data.trim_trace(df_llc_orig, df_loss, task)
    vis.plot_data(fig, *full_curve, color=faded, mode="lines")
    interval = data.trim_trace(df_llc_orig, df_loss, task, *analysis_interval)
    vis.plot_data(fig, *interval, color=col, name=f"{task}-original", mode="lines+markers")

    
    new_curve = data.trim_trace(df_llc_nb10, df_loss, task)
    vis.plot_data(fig, *new_curve, color=faded, mode="lines")
    new_interval = data.trim_trace(df_llc_nb10, df_loss, task, *analysis_interval)
    vis.plot_data(fig, *new_interval, color=col, name=f"{task}-nbeta=10", mode="lines+markers")
    
fig.update_xaxes(title_text=vis.llc_desc)
fig.update_yaxes(title_text=vis.loss_desc)
fig.update_layout(width=1000, height=800, title=f"Pythia {model} Original and with nbeta=10")
fig.show()

# Fit powerlaws

In [ ]:
fig = go.Figure()
report = []  # store parameters

for task in tasks:
    color = colors[task]
    llc, loss, step = trace = data.trim_trace(df_llc_nb10, df_loss, task, *analysis_interval)
    result = fit.min_fit(llc, loss, fit.OffsetPowerLaw)
    shift = None  # set to result.params_dict if you want Dan's transformed style
    
    vis.plot_data(fig, *trace, color=color, shift=shift)
    vis.plot_result(fig, llc, result, color=color, name=task,
                    showlegend=True, shift=shift)

    report_row = {
        "dataset": task,
        "L*": result.params_dict["y*"],
        "r": result.params_dict["r"],
        "fit_r2": fit.r2_score(loss, result.f(llc))
    }   
    

    # Compare with original on same interval
    llc, loss, step = trace = data.trim_trace(df_llc_orig, df_loss, task, *analysis_interval)
    result_o = fit.min_fit(llc, loss, fit.OffsetPowerLaw)
    report_row.update({
        "Original L*": result_o.params_dict["y*"],
        "Original r": result_o.params_dict["r"],
        #"Original fit_r2": fit.r2_score(loss, result_o.f(llc))
    })
    
    report.append(report_row)

fig.update_layout(width=800, height=600, title=f"Pythia-{model} Power Laws with nbeta=10")
fig.update_xaxes(title_text=vis.llc_desc)
fig.update_yaxes(title_text=vis.loss_desc)
fig.show()
pd.DataFrame(report)

In [ ]:
# It occurs to me,
# The loss vs step is the same, and fits pretty well to a power law itself
# what we're seeing in common is a logarithmic growth in LLC through this analysis interval

# Just plot the differences?
analysis_interval = 2000, 70000

fig = go.Figure()
tasks = df_llc_nb10.columns
colors = vis.assign_cols(tasks)

# Something wrong here

for task in tasks:
    col = colors[task]
    faded = vis.fade(col)
    
    # llc, loss, step = dat.trim_trace(df_llc_orig, df_loss, task, *analysis_interval)
    # vis.plot_data(fig, step, llc, step, color=faded, name=f"{task}-original", mode="lines+markers")
    llc, loss, step = data.trim_trace(df_llc_nb10, df_loss, task)
    vis.plot_data(fig, step, llc, step, color=faded, mode="lines+markers")
    
    llc, loss, step = data.trim_trace(df_llc_nb10, df_loss, task, *analysis_interval)
    vis.plot_data(fig, step, llc, step, color=col, name=f"{task}-nbeta=10", mode="lines+markers")
    
fig.update_xaxes(title_text="step")
fig.update_yaxes(title_text=vis.llc_desc)
fig.update_layout(
    title = "LLC vs time",
    xaxis=dict(type='log'),
    width=800,
    height=600,
    #yaxis=dict(type='log'),
)
fig.show()
